In [1]:
%load_ext autoreload
%autoreload 2

import os
import subprocess
import sys
import warnings

warnings.simplefilter("ignore", FutureWarning)

import xclone_config
project_config = xclone_config
os.chdir(project_config.ROOT)

from collections import defaultdict, OrderedDict
import multiprocessing as mp
import sys

from joblib import Parallel, delayed
import numpy as np
import pandas as pd
from tqdm import tqdm, tqdm_notebook

import matplotlib.pyplot as plt
import seaborn as sns

from data_types import ase
import plotlib
import toolkit
import util
import test_phasing
from workspace.workspace_manager import WorkspaceManager
from data_types import ase

workspace = WorkspaceManager(
    task_name="preprocessing",
    experiment_info={"sample" : "N5CC3E", 
                     "modality" : "scATAC"},
    verbose=True
)
workspace.load_workspace()

sns.set()

In [3]:
data = {
    data_type : util.pickle_load(workspace.tmp_data[data_type])
    for data_type in ["snp_counts", "phasing", "blocks"]
}
data["snp"] = data["snp_counts"]\
                [["CHROM", "POS"]]\
                .to_dense()\
                .astype(np.int64)

KeyError: 'snp_counts'

In [ ]:
chrom_to_blocks = {
    str(chrom) : data["blocks"][data["blocks"]["CHROM"] == chrom]
    for chrom in data["blocks"]["CHROM"].unique()
}

In [ ]:
%%time

# Here block coverage for each phased SNP is computed
# TODO: rewrite this using "bedtools intersect".
# This part doesn't scale well.

def intersect_snp_with_blocks(snp_tuple):
    chrom, pos = snp_tuple
    # 1-based to 0-based 
    pos -= 1 # because CellSNP is 1-based, but .bed files are 0-based
    blocks_on_chrom = chrom_to_blocks.get(str(chrom), None)
    if blocks_on_chrom is None:
        return ""
    mask = ((blocks_on_chrom.START <= pos) 
            & (pos < blocks_on_chrom.END))
    return '@'.join(blocks_on_chrom[mask].BLOCK_ID)

# result = [intersect_snp_with_blocks(snp_tuple) 
#           for snp_tuple in tqdm_notebook(data["snp"][data["snp"]["CHROM"] == 10].values, 
#                                          "SNP processing")]
pool = mp.Pool(16)
result = pool.map(intersect_snp_with_blocks, 
                  tqdm_notebook(data["snp"].values, 
                                "SNP processing"))
pool.close()
pool.join()

In [ ]:
""" Here the raw results computed in parallel are parsed """
snp_to_blocks, block_to_snps = {}, OrderedDict()

# This way we ensure proper block ordering
for block_id in data["blocks"].BLOCK_ID:
    block_to_snps[block_id] = []

for i, row in tqdm_notebook(enumerate(data["snp"].values), 
                            "mapping SNPs to blocks"):
    chrom, pos = row
    snp = f"{chrom},{pos}"
    snp_to_blocks[snp] = result[i].split("@") if result[i] else []
    for block in snp_to_blocks[snp]:
        block_to_snps[block].append(snp)

In [ ]:
plt.figure(figsize=(16,8))
sns.set(font_scale=1.5)
plt.title(f"{workspace.experiment_info['modality']}, "
          "number of blocks covering one SNP")
plt.xlabel("number of blocks")
plt.ylabel("snp count")
plt.hist([len(block_list) for block_list in snp_to_blocks.values()]);

In [ ]:
plt.figure(figsize=(16,8))
plt.title(f"{workspace.experiment_info['modality']}"
          ", number of SNPs covered by block")
plt.xlabel("number of snps")
plt.ylabel("block count")
plt.hist([len(snp_list) for snp_list 
          in block_to_snps.values()]);

In [ ]:
util.pickle_dump(
    snp_to_blocks, 
    f"{workspace.tmp_dir}/snp_to_blocks.pkl"
)
workspace.add_entry("snp_to_blocks", "snp_to_blocks.pkl")
util.pickle_dump(
    block_to_snps, 
    f"{workspace.tmp_dir}/block_to_snps.pkl"
)
workspace.add_entry("block_to_snps", "block_to_snps.pkl")

In [ ]:
workspace.status()

In [ ]:
snp_to_idx = {snp : i for i, snp in 
              tqdm_notebook(enumerate(toolkit.extract_snps(data["snp_counts"])), 
                            "mapping snps to their index numbers "
                            "by position in the blocks")}

In [ ]:
%%time

block_to_snp_ids = {block : np.array([snp_to_idx[snp] for snp in snp_list]) 
                   for block, snp_list 
                   in tqdm_notebook(block_to_snps.items(), 
                                    desc="mapping block to SNP ids"
                                    " (for faster row selection)")}

import numba
# @numba.jit(nopython=True)
def get_block_counts(ad, dp, snp_ids):
    if snp_ids.size > 0:
#         mask = np.zeros(ad.size).astype(np.bool)
#         mask[snp_ids] = True
#         ad_sample, dp_sample = ad[s], dp[mask]
        return np.nansum(ad[snp_ids]), np.nansum(dp[snp_ids])
    return np.nan, np.nan


def extract_block_counts(barcode):
    block_to_ad, block_to_dp = [], []
    # I need to convert these columns to dense format
    # because I need only a subset of rows on each iteration 
    ad = np.array(data["snp_counts"][f"{barcode}_ad"])
    dp = np.array(data["snp_counts"][f"{barcode}_dp"])
    
    # block_to_snps is an OrderedDict, so we can guarantee
    # that all the blocks are processed in a correct order
    for snp_ids in block_to_snp_ids.values():
        if len(snp_ids) > 0:
            dp_sum = np.nansum(dp[snp_ids])
            block_to_dp.append(np.nan if dp_sum == 0 else dp_sum)
            block_to_ad.append(np.nan if dp_sum == 0 
                               else np.nansum(ad[snp_ids]))
    
    return pd.DataFrame({f"{barcode}_ad" : block_to_ad, 
                         f"{barcode}_dp" : block_to_dp})


pool = mp.Pool(16)
result_list = pool.map(
    extract_block_counts, 
    tqdm_notebook(toolkit.extract_barcodes(data["snp_counts"]), 
                  desc="cell_barcode processing")
)
pool.close()
pool.join()
# result_list = [extract_block_counts(barcode) for barcode in 
#                tqdm_notebook(toolkit.extract_barcodes(data["snp_counts"]), 
#                              desc="cell_barcode processing")]
data["block_counts"] = pd.concat(result_list, axis=1)
data["block_counts"].info()

In [ ]:
data["block_counts"].insert(
    0, "BLOCK_ID", 
    [block for block, snp_list 
     in block_to_snps.items() 
     if len(snp_list) > 0]
)
data["block_counts"].head()

In [ ]:
# data["clustering"] = util.pickle_load(f"{workspace.tmp_dir}/clustering.pkl")

In [ ]:
# clean_barcodes = data["clustering"]["BARCODE"]
# raw_barcodes = toolkit.extract_barcodes(data["block_counts"])
# noisy_barcodes = np.setdiff1d(raw_barcodes, clean_barcodes)
# data["block_counts"].drop(
#     columns=[f"{barcode}_ad" for barcode in noisy_barcodes],
#     inplace=True
# )
# data["block_counts"].drop(
#     columns=[f"{barcode}_dp" for barcode in noisy_barcodes],
#     inplace=True
# )

In [ ]:
remaining_blocks = data["blocks"][
    data["blocks"].BLOCK_ID.isin(data["block_counts"].BLOCK_ID)
].reset_index(drop=True)

data["block_counts"] = data["block_counts"].merge(
    remaining_blocks,
    on="BLOCK_ID"
)

In [ ]:
data["block_counts"] = data["block_counts"].to_sparse()

In [ ]:
util.pickle_dump(
    data["block_counts"],
    os.path.join(
        workspace.tmp_dir,
        "block_counts_T1.pkl"
    )
)
workspace.add_entry("block_counts_T1", "block_counts_T1.pkl")
workspace.verify()
workspace.push()

In [ ]:
data["ase"] = ase.compute_ase(
    data["block_counts"], 
    toolkit.extract_barcodes(data["block_counts"])
).to_sparse()

In [ ]:
data["ase"].info()

In [ ]:
util.pickle_dump(
    data["ase"],
    os.path.join(
        workspace.tmp_dir,
        "ase_T1.pkl"
    )
)
workspace.add_entry("ase_T1", "ase_T1.pkl")
workspace.verify()
workspace.push()

In [ ]:
data["block_counts"][["BLOCK_ID", "CHROM", "START", "END"]].to_csv(
    os.path.join(
        workspace.dir,
        "annotation_T1.csv"
    ), index=False
)
with open(os.path.join(workspace.dir, "barcodes_T1.txt"), "w") as outfile:
    outfile.write("\n".join(toolkit.extract_barcodes(data["block_counts"])))